# Example: Potjans & Diesmann microcircuit  2014

This example notebook submits a job to a chosen HPC backend of EBRAINS/HBP (**JUSUF**), loads a **Singularity** container with a complete **hybridLFPy** installation and executes the hybridLFPy example file ``example_microcircuit.py`` (https://github.com/INM-6/hybridLFPy/blob/master/examples/example_microcircuit.py) in parallel on the backend.

For more in-depth info on using HPC backends via EBRAINS see https://wiki.ebrains.eu/bin/view/Collabs/using-supercomputers-from-the-collab/

## Prepare connection to backend

In [2]:
# use the pyunicore library
!pip install pyunicore==0.9.12 --upgrade

Requirement already up-to-date: pyunicore==0.9.12 in /opt/app-root/lib/python3.6/site-packages
Requirement already up-to-date: PyJWT>=1.7 in /opt/app-root/lib/python3.6/site-packages (from pyunicore==0.9.12)
Requirement already up-to-date: requests>=2.5 in /opt/app-root/lib/python3.6/site-packages (from pyunicore==0.9.12)
Requirement already up-to-date: urllib3<1.27,>=1.21.1 in /opt/app-root/lib/python3.6/site-packages (from requests>=2.5->pyunicore==0.9.12)
Requirement already up-to-date: charset-normalizer~=2.0.0; python_version >= "3" in /opt/app-root/lib/python3.6/site-packages (from requests>=2.5->pyunicore==0.9.12)
Requirement already up-to-date: certifi>=2017.4.17 in /opt/app-root/lib/python3.6/site-packages (from requests>=2.5->pyunicore==0.9.12)
Requirement already up-to-date: idna<4,>=2.5; python_version >= "3" in /opt/app-root/lib/python3.6/site-packages (from requests>=2.5->pyunicore==0.9.12)
You are using pip version 9.0.1, however version 21.3.1 is available.
You should c

In [3]:
# import modules
import os
import pyunicore.client as unicore_client
import requests
import json
from time import sleep, time
from pprint import pprint
from IPython.display import IFrame, clear_output

In [4]:
# Create connection to supercomputer (i.e., JUSUF)
tr = unicore_client.Transport(clb_oauth.get_token())
r = unicore_client.Registry(tr, unicore_client._HBP_REGISTRY_URL)

In [5]:
# Valid choices for supercomputers are one of the keys in:
r.site_urls

{'JUWELS': 'https://zam2125.zam.kfa-juelich.de:9112/JUWELS/rest/core',
 'JUSUF': 'https://zam2125.zam.kfa-juelich.de:9112/JUSUF/rest/core',
 'UNICORE-TEST': 'https://catto.cscs.ch:8080/UNICORE-TEST/rest/core',
 'BSC-MareNostrum': 'https://unicore-hbp.bsc.es:8080/BSC-MareNostrum/rest/core',
 'JURECA': 'https://zam2125.zam.kfa-juelich.de:9112/JURECA/rest/core',
 'DAINT-CSCS': 'https://brissago.cscs.ch:8080/DAINT-CSCS/rest/core',
 'CINECA-MARCONI': 'https://grid.hpc.cineca.it:9111/CINECA-MARCONI/rest/core',
 'CINECA-G100': 'https://grid.hpc.cineca.it:9111/CINECA-G100/rest/core',
 'CINECA-M100': 'https://grid.hpc.cineca.it:9111/CINECA-M100/rest/core',
 'JUDAC': 'https://zam2125.zam.kfa-juelich.de:9112/JUDAC/rest/core'}

In [6]:
# choose one HPC resource
supercomputer = 'JUSUF'
try:
    site_client = r.site(supercomputer)
except KeyError:
    # cluster may be dropped from Registry.site_urls for whatever reason
    site_client = unicore_client.Client(r.transport, 
                                        'https://zam2125.zam.kfa-juelich.de:9112/JUSUF/rest/core')

In [7]:
site_client.access_info()

{'role': {'selected': 'user', 'availableRoles': ['user']},
 'queues': {'availableQueues': ['batch', 'develgpus'], 'selected': 'batch'},
 'dn': 'UID=espen.hagen@nmbu.no',
 'xlogin': {'UID': 'hagen2',
  'availableGroups': ['icei-hbp-2020-0004'],
  'availableUIDs': ['hagen2'],
  'group': 'icei-hbp-2020-0004'}}

In [8]:
# override group entry in case of multiple allocations
# site_client.transport.preferences = "group:icei-hbp-2020-0004"
# site_client.access_info()

In [9]:
# check connection to supercomputer
headers = {}
headers["Authorization"] = "Bearer " + clb_oauth.get_token()
headers['Accept'] = "application/json"
rs = requests.get(url=site_client.site_url, headers = headers, verify = False)
print("Status code %s " % rs.status_code)
print("Content-type %s " % rs.headers['Content-Type'])
reply = rs.json()
# print(json.dumps(reply, indent = 1))

Status code 200 
Content-type application/json 


## Prepare singularity container
See https://apps.fz-juelich.de/jsc/hps/jusuf/cluster/container-runtime.html#container-runtime-on-system-name for details. 

This step builds the singularity container. It is optional if the recipe has already been uploaded and built on the system. 

The procedure may be different on different HPC backends. 
The container can either way be built from the same recipe: https://raw.githubusercontent.com/INM-6/hybridLFPy/master/Dockerfile

Turn the below Raw block into Code in order to run:

## Prepare main simulation job
This step combines in a single session the following:

- download hybridLFPy container
- download simulation files from GitHub (https://github.com/INM-6/hybridLFPy)
- ask for resources (# nodes, # MPI processes, # seconds runtime)
- execute simulation via the batch job queue
- download tar file with simulation output

### Notes

Singularity exposes the host environment to the running container (i.e., ``$PATH``, ``$PYTHONPATH``, ``$HOME`` etc.). 
The container includes Python 3.8.5, and may find pip-installed software in your host's ``$HOME/.local/lib/python3.8/site-packages`` directory and similar if it exist. 

If you encounter problems, try including ``unset PYTHONPATH`` in your ``simulation_job["Executable"]`` part below.

In [10]:
# create dictionary with job specification and define list of input files from this Collab
# simulation_job = {"Job type": "interactive"}
simulation_job = {}
simulation_inputs = []

In [11]:
# Resources
simulation_job['Resources'] = {
    "Queue": "batch",
    "Nodes": "4",
    "CPUsPerNode": "128",
    #"CPUs": "512",
    "Runtime": "3600",
}

In [12]:
# commands run on login node before job execution
simulation_job["User precommand"] = """module use $OTHERSTAGES
module --force purge
module load Stages/2020
module load GCC Singularity-Tools git

git clone --branch master --depth 1 https://github.com/INM-6/hybridLFPy.git
cd hybridLFPy/examples
sib download --recipe-name hybridlfpy
"""
simulation_job["RunUserPrecommandOnLoginNode"] = "true"

In [13]:
# - set some environment variables
# - run the python code using interpreter embedded in container
simulation_job["Executable"] = """module use $OTHERSTAGES
module --force purge
module load Stages/2020
module load GCC Singularity-Tools
unset DISPLAY  # matplotlib may look for a nonexistant display on compute node(s)
export SINGULARITYENV_SCRATCH=  # make container unaware of $SCRATCH area as we are already there - write sim output to simulation script folder
export SINGULARITYENV_PYTHONPATH=/opt/nest/lib/python3.9/site-packages
cd hybridLFPy/examples  # go to examples
srun --mpi=pmi2 singularity exec hybridlfpy.sif python3 -u example_microcircuit.py  # execute simulation
"""

In [14]:
# commands run after job is done
simulation_job["User postcommand"] = "" 
simulation_job["RunUserPostcommandOnLoginNode"] = "true"

In [15]:
simulation_job

{'Resources': {'Queue': 'batch',
  'Nodes': '4',
  'CPUsPerNode': '128',
  'Runtime': '3600'},
 'User precommand': 'module use $OTHERSTAGES\nmodule --force purge\nmodule load Stages/2020\nmodule load GCC Singularity-Tools git\n\ngit clone --branch master --depth 1 https://github.com/INM-6/hybridLFPy.git\ncd hybridLFPy/examples\nsib download --recipe-name hybridlfpy\n',
 'RunUserPrecommandOnLoginNode': 'true',
 'Executable': 'module use $OTHERSTAGES\nmodule --force purge\nmodule load Stages/2020\nmodule load GCC Singularity-Tools\nunset DISPLAY  # matplotlib may look for a nonexistant display on compute node(s)\nexport SINGULARITYENV_SCRATCH=  # make container unaware of $SCRATCH area as we are already there - write sim output to simulation script folder\nexport SINGULARITYENV_PYTHONPATH=/opt/nest/lib/python3.9/site-packages\ncd hybridLFPy/examples  # go to examples\nsrun --mpi=pmi2 singularity exec hybridlfpy.sif python3 -u example_microcircuit.py  # execute simulation\n',
 'User postc

In [16]:
# create job
job = site_client.new_job(job_description=simulation_job, inputs=simulation_inputs)

In [17]:
job.working_dir

Storage: https://zam2125.zam.kfa-juelich.de:9112/JUSUF/rest/core/storages/da6e75a2-fca4-4a94-9869-48fee1466639-uspace

In [18]:
print(json.dumps(job.properties['log'], indent = 1))

[
 "Mon Nov 22 08:58:35 CET 2021: Created with ID da6e75a2-fca4-4a94-9869-48fee1466639",
 "Mon Nov 22 08:58:35 CET 2021: Created with type 'JSON'",
 "Mon Nov 22 08:58:35 CET 2021: Client: Name: UID=espen.hagen@nmbu.no\nXlogin: uid: [hagen2], gids: [icei-hbp-2020-0004, addingOSgroups: true]\nRole: user: role from attribute source\nQueues: [batch:develgpus, selected=batch]\nSecurity tokens: User: UID=espen.hagen@nmbu.no\nClient's original IP: 134.94.88.93",
 "Mon Nov 22 08:58:37 CET 2021: No staging in needed.",
 "Mon Nov 22 08:58:37 CET 2021: Status set to READY.",
 "Mon Nov 22 08:58:37 CET 2021: Status set to PENDING.",
 "Mon Nov 22 08:58:37 CET 2021: Launched pre command <module use $OTHERSTAGES\nmodule --force purge\nmodule load Stages/2020\nmodule load GCC Singularity-Tools git\n\ngit clone --branch master --depth 1 https://github.com/INM-6/hybridLFPy.git\ncd hybridLFPy/examples\nsib download --recipe-name hybridlfpy\n>"
]


In [19]:
# wait while job is running and track progress
while job.is_running():
    clear_output()
    # STDERR output (if any)
    try:
        stderr = job.working_dir.stat('stderr')
        #pprint(stderr.raw().readlines()[:5])
        pprint(stderr.raw().readlines()[-5:])
    except:
        print('no stderr file to read yet')
    # STDOUT output
    try:
        stdout = job.working_dir.stat('stdout')
        #pprint(stdout.raw().readlines()[:10])
        pprint(stdout.raw().readlines()[-10:])
    except:
        print('no stdout file to read yet')
    sleep(10)

[b'--No graphics will be displayed.\n',
 b'Translating alphaisyn.mod into /p/scratch/icei-hbp-2020-0004/unicore-jobs/da'
 b'6e75a2-fca4-4a94-9869-48fee1466639/hybridLFPy/examples/x86_64/alphaisyn.c\n',
 b'Translating expsyni.mod into /p/scratch/icei-hbp-2020-0004/unicore-jobs/da6e'
 b'75a2-fca4-4a94-9869-48fee1466639/hybridLFPy/examples/x86_64/expsyni.c\n',
 b'Thread Safe\n',
 b'Thread Safe\n']
[b'Execution time: 721.552 seconds\n',
 b'Execution time: 720.841 seconds\n',
 b'Execution time: 720.590 seconds\n',
 b'Execution time: 720.536 seconds\n',
 b'Execution time: 721.579 seconds\n',
 b'Execution time: 720.964 seconds\n',
 b'Execution time: 720.819 seconds\n',
 b'Execution time: 720.546 seconds\n',
 b'Execution time: 720.542 seconds\n',
 b'Execution time: 720.586 seconds\n']


In [22]:
job.working_dir.listdir().keys()

dict_keys(['UNICORE_SCRIPT_EXIT_CODE', 'stderr', 'hybridLFPy/', 'bss_submit_1637567975267', 'stdout', '.UNICORE_POST_0/', '.UNICORE_PRE_0/', 'UNICORE_Job_1637567975267'])

In [23]:
# download simulation output
job.working_dir.stat('hybridLFPy/examples/simulation_output_example_microcircuit.tar'
                     ).download('simulation_output_example_microcircuit.tar')

In [24]:
# kill job, clean up files on the remote
job.delete()

## Investigate simulation output
Here we choose to print some pdf file output from the simulation, however all simulation output are available within the same simulation output file hierarchy.

In [27]:
# remove old simulation output (if present) and untar output to folder example_network_output
!rm -r simulation_output_example_microcircuit
!tar -xf simulation_output_example_microcircuit.tar

In [28]:
IFrame("./simulation_output_example_microcircuit/figures/layers.pdf", width=800, height=600)

In [29]:
# quick look at e.g., the extracellular potential and spike raster plot
IFrame("./simulation_output_example_microcircuit/figures/cell_models.pdf", width=800, height=600)

In [30]:
IFrame("./simulation_output_example_microcircuit/figures/soma_locations.pdf", width=800, height=600)

In [31]:
IFrame("./simulation_output_example_microcircuit/figures/network_raster.pdf", width=800, height=600)

In [32]:
IFrame("./simulation_output_example_microcircuit/figures/compound_signals.pdf", width=800, height=600)

In [33]:
IFrame("./simulation_output_example_microcircuit/figures/current_dipole_moments.pdf", width=800, height=600)